In [ ]:
!pip install ultralytics

In [ ]:
#requirements
import glob
import shutil
import yaml
import os
import re
from tqdm import tqdm
import random
import ultralytics
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/cj_data'

In [ ]:
#Unzip Files
# %cd /content/drive/MyDrive/cj_data
# #!unzip "Data.zip"
# !unzip "labels.zip"
# %cd /content

In [ ]:
image_len = (glob.glob(f'{path}/data/image/*.jpg'))
len(image_len)

596

In [ ]:
labels_len = (glob.glob(f'{path}/labels/*.txt'))
len(labels_len)

589

In [ ]:
validation_len = (glob.glob(f'{path}/data/validation/**/*.jpg'))
len(validation_len)


45

### making Yaml File

In [ ]:
with open(f'{path}/data/labeling.txt', 'r', encoding='utf-8') as la:
    next(la)
    lines = la.readlines()
    classes = [(line.strip().split('\t')) for line in lines]


In [ ]:
labels_lst = []
labels = []
for cl in classes:
    labels_lst.append((str(cl[0])+ '_' + cl[1]))
    labels.append(cl[0])

In [ ]:
data_yaml = {
    'nc':len(classes),
    'names': labels,
    'train': '/content/drive/MyDrive/cj_data/data/train',
    'val': '/content/drive/MyDrive/cj_data/data/val',
    'test': '/content/drive/MyDrive/cj_data/data/test'
}
with open(f'{path}/data/data.yaml', 'w', encoding='utf-8') as f:
    yaml.dump(data_yaml, f,allow_unicode=True)

In [ ]:
!cat /content/drive/MyDrive/cj_data/Data/data.yaml